In [97]:
from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
from selenium.common.exceptions import InvalidArgumentException
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [98]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh/robots.txt')
rp.read()

url = 'https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh'
rp.can_fetch('*', url)

False

In [99]:
def crawl_one_dog(link):
    print('https://www.chotot.com' + link)
    driver = webdriver.Firefox()
    try:
        driver.get('https://www.chotot.com' + link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        price = np.nan
        description = np.nan
        breed = np.nan        
        age = np.nan        
        size = np.nan
        address = np.nan
        # phone_num = np.nan

        price_tag = soup.find('span', attrs = {'itemprop': 'price'})
        price = price_tag.text if price_tag else np.nan

        description_tag = soup.find('p', attrs = {'class': 'AdDecription_adBody__qp2KG'})
        description = description_tag.text if description_tag else np.nan

        breed_tag = soup.find('span', attrs = {'itemprop': 'pet_breed'})
        breed = breed_tag.text if breed_tag else np.nan

        age_tag = soup.find('span', attrs = {'itemprop': 'pet_age'})
        age = age_tag.text if age_tag else np.nan

        size_tag = soup.find('span', attrs = {'itemprop': 'pet_size'})
        size = size_tag.text if size_tag else np.nan

        address_tag = soup.find('div', attrs={'class':'media-body media-middle AdParam_address__5wp1F'})
        address = address_tag.text if address_tag else np.nan

        # # Click the button to show the phone number
        # driver.execute_script("var element = document.querySelector('.aw__s1cdo2zu'); if (element) element.parentNode.removeChild(element);")
        # button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ShowPhoneButton_icon__wsnZ5')))
        # driver.execute_script("arguments[0].scrollIntoView();", button)
        # button.click()
        # soup = BeautifulSoup(driver.page_source, 'html.parser')
        # # Wait for the phone number to load (adjust the timeout as needed)
        # phone_num_tag = soup.find('div', attrs = {'class': 'ShowPhoneButton_phoneButton__p5Cvt ShowPhoneButton_phoneClicked__IxuR6'})
        # phone_num = phone_num_tag.text if phone_num_tag else np.nan

        driver.quit()
        return price, description, breed, age, size, address
    except InvalidArgumentException:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    finally:
        driver.quit()

In [100]:
df = pd.DataFrame()
param = '?page='
# index_page = 123
index_page = 1
while(True):
    driver = webdriver.Firefox()
    try:
        driver.get(url + param + str(index_page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        not_found = soup.find('div', attrs={'class': 'NotFound_content__KtIbC'})
        if not_found is not None:
            print('No more dogs to crawl')
            break

        dogs_links = soup.find_all('div', attrs={'role' : 'button'})
        for i in dogs_links:
            price, description, breed, age, size, address = crawl_one_dog(i.find('a')['href'])
            
            data = pd.DataFrame({
                'price': [price],
                'description': [description],
                'breed': [breed],
                'age': [age],
                'size': [size],
                'address': [address],
                # 'phone_num': [phone_num]
            })
            df = pd.concat([df, data], axis = 0)
    finally:
        driver.quit()

    index_page += 1

No more dogs to crawl


In [101]:
df.to_csv('Dog.csv', index = False)